In [4]:
%pip install scikit-multilearn

You should consider upgrading via the '/bin/python3 -m pip install --upgrade pip' command.
Note: you may need to restart the kernel to use updated packages.


In [5]:
#import sys
import numpy as np
import pandas as pd
%matplotlib inline
#import matplotlib.pyplot as plt
#import seaborn as sns
#import sklearn
import tensorflow as tf
import es_utils as utils

In [6]:
import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  2


In [7]:
'''labels = ['label:LYING_DOWN',
 'label:SITTING',
 'label:FIX_walking',
 'label:FIX_running',
 'label:BICYCLING',
 'label:SLEEPING',
 'label:LAB_WORK',
 'label:IN_CLASS',
 'label:IN_A_MEETING',
 'label:LOC_main_workplace',
 'label:OR_indoors',
 'label:OR_outside',
 'label:IN_A_CAR',
 'label:ON_A_BUS',
 'label:DRIVE_-_I_M_THE_DRIVER',
 'label:DRIVE_-_I_M_A_PASSENGER',
 'label:LOC_home',
 'label:FIX_restaurant',
 'label:PHONE_IN_POCKET',
 'label:OR_exercise',
 'label:COOKING',
 'label:SHOPPING',
 'label:STROLLING',
 'label:DRINKING__ALCOHOL_',
 'label:BATHING_-_SHOWER',
 'label:CLEANING',
 'label:DOING_LAUNDRY',
 'label:WASHING_DISHES',
 'label:WATCHING_TV',
 'label:SURFING_THE_INTERNET',
 'label:AT_A_PARTY',
 'label:AT_A_BAR',
 'label:LOC_beach',
 'label:SINGING',
 'label:TALKING',
 'label:COMPUTER_WORK',
 'label:EATING',
 'label:TOILET',
 'label:GROOMING',
 'label:DRESSING',
 'label:AT_THE_GYM',
 'label:STAIRS_-_GOING_UP',
 'label:STAIRS_-_GOING_DOWN',
 'label:ELEVATOR',
 'label:OR_standing',
 'label:AT_SCHOOL',
 'label:PHONE_IN_HAND',
 'label:PHONE_IN_BAG',
 'label:PHONE_ON_TABLE',
 'label:WITH_CO-WORKERS',
 'label:WITH_FRIENDS']

len(labels)'''

labels = [
    'label:OR_standing',
    'label:SITTING',
    'label:LYING_DOWN',
    'label:FIX_running',
    'label:FIX_walking',
    'label:BICYCLING'
]


In [8]:
tf.__version__

'2.9.1'

In [10]:
paths = ['/home/nonroot/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            'neurons_1' : 32, 
            'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    test_results, ba = har.run()
    #model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
print(max(bas))


X train-test shape: (3168, 226) - X test shape: (792, 226)
y train-test shape: (3168, 6) - y test shape: (792, 6)
Epoch 1/40
254/254 [==============================] - 1s 2ms/step - loss: 1.0766 - val_loss: 1.0891
Epoch 2/40
254/254 [==============================] - 0s 2ms/step - loss: 1.0253 - val_loss: 1.1346
Epoch 3/40
254/254 [==============================] - 0s 2ms/step - loss: 1.1797 - val_loss: 1.2915
Epoch 4/40
254/254 [==============================] - 0s 1ms/step - loss: 1.4185 - val_loss: 1.5032
Epoch 5/40
254/254 [==============================] - 0s 2ms/step - loss: 1.6630 - val_loss: 2.1367
Epoch 6/40
254/254 [==============================] - 0s 2ms/step - loss: 1.9819 - val_loss: 1.7339
Epoch 7/40
254/254 [==============================] - 0s 2ms/step - loss: 2.6647 - val_loss: 2.4266
Epoch 8/40
254/254 [==============================] - 0s 2ms/step - loss: 4.4053 - val_loss: 2.9900
Epoch 9/40
254/254 [==============================] - 0s 1ms/step - loss: 2.7494 - val

In [11]:
'''def nan_mse(y_actual, y_predicted):
    stack = tf.stack((tf.math.is_nan(y_actual), 
                      tf.math.is_nan(y_predicted)),
                     axis=1)
    is_nans = tf.keras.backend.any(stack, axis=1)
    per_instance = tf.where(is_nans,
                            tf.zeros_like(y_actual),
                            tf.square(tf.subtract(y_predicted, y_actual)))
    print(stack)
    print(per_instance)
    return tf.reduce_mean(per_instance, axis=0)

print(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'''

'def nan_mse(y_actual, y_predicted):\n    stack = tf.stack((tf.math.is_nan(y_actual), \n                      tf.math.is_nan(y_predicted)),\n                     axis=1)\n    is_nans = tf.keras.backend.any(stack, axis=1)\n    per_instance = tf.where(is_nans,\n                            tf.zeros_like(y_actual),\n                            tf.square(tf.subtract(y_predicted, y_actual)))\n    print(stack)\n    print(per_instance)\n    return tf.reduce_mean(per_instance, axis=0)\n\nprint(nan_mse([1.,1.,np.nan,1.,0.], [1.,1.,0.,0.,np.nan]))'

In [12]:
paths = ['/home/nonroot/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv', 
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv',
         '/home/nonroot/sample_data/0BFC35E2-4817-4865-BFA7-764742302A2D.features_labels.csv']
bas = []

for path in paths:

    config = {
            'df_path': path,
            #'df_path': '../full_data/exp_/fold_0/raw_40.csv',
            #'neurons_1' : 32, 
            #'neurons_2' : 16, 
            'labels': labels
    }

    har = utils.HAR(config)
    #test_results, ba = har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)

X train-test shape: (3168, 226) - X test shape: (792, 226)
y train-test shape: (3168, 6) - y test shape: (792, 6)
INFO:tensorflow:Reloading Oracle from existing project hypertunning/experimento_1_RandSearchCV/oracle.json
INFO:tensorflow:Reloading Tuner from hypertunning/experimento_1_RandSearchCV/tuner0.json
INFO:tensorflow:Oracle triggered exit
Epoch 1/100
51/51 [==============================] - 1s 8ms/step - loss: 0.3546 - avg_multilabel_BA_2: 0.6056 - val_loss: 0.1193 - val_avg_multilabel_BA_2: 0.6740
Epoch 2/100
51/51 [==============================] - 0s 3ms/step - loss: 0.0998 - avg_multilabel_BA_2: 0.7167 - val_loss: 0.1021 - val_avg_multilabel_BA_2: 0.7492
Epoch 3/100
51/51 [==============================] - 0s 3ms/step - loss: 0.0922 - avg_multilabel_BA_2: 0.7721 - val_loss: 0.1037 - val_avg_multilabel_BA_2: 0.7912
Epoch 4/100
51/51 [==============================] - 0s 3ms/step - loss: 0.0820 - avg_multilabel_BA_2: 0.8058 - val_loss: 0.1278 - val_avg_multilabel_BA_2: 0.8185


In [13]:

bas = []
config = {
        #'df_path': path,
        'df_path': './sample_data/exp_/fold_0/raw_40.csv',
        'neurons_1' : 32, 
        'neurons_2' : 16, 
        'labels': labels
}

har = utils.HAR(config)
test_results, ba = har.run()
#model, best_hps, best_epoch, test_results, ba = har.hypertunning()
bas.append(ba.numpy())


print(bas)
print(max(bas))

X train-test shape: (242349, 226) - X test shape: (60588, 226)
y train-test shape: (242349, 6) - y test shape: (60588, 6)
Epoch 1/40
19388/19388 [==============================] - 29s 1ms/step - loss: 86.2202 - val_loss: 183.3051
Epoch 2/40
19388/19388 [==============================] - 28s 1ms/step - loss: 338.0593 - val_loss: 150.8320
Epoch 3/40
19388/19388 [==============================] - 28s 1ms/step - loss: 735.2804 - val_loss: 975.8772
Epoch 4/40
19388/19388 [==============================] - 29s 1ms/step - loss: 1177.1965 - val_loss: 3403.5513
Epoch 5/40
19388/19388 [==============================] - 28s 1ms/step - loss: 2030.8647 - val_loss: 541.6123
Epoch 6/40
19388/19388 [==============================] - 28s 1ms/step - loss: 2488.1743 - val_loss: 4141.4243
Epoch 7/40
19388/19388 [==============================] - 29s 1ms/step - loss: 3185.8894 - val_loss: 10478.1699
Epoch 8/40
19388/19388 [==============================] - 29s 1ms/step - loss: 5489.1313 - val_loss: 2757.80

In [14]:
har.data.x.head()

,raw_acc:magnitude_stats:mean,raw_acc:magnitude_stats:std,raw_acc:magnitude_stats:moment3,raw_acc:magnitude_stats:moment4,raw_acc:magnitude_stats:percentile25,raw_acc:magnitude_stats:percentile50,raw_acc:magnitude_stats:percentile75,raw_acc:magnitude_stats:value_entropy,raw_acc:magnitude_stats:time_entropy,raw_acc:magnitude_spectrum:log_energy_band0,...,lf_measurements:temperature_ambient,discrete:time_of_day:between0and6,discrete:time_of_day:between3and9,discrete:time_of_day:between6and12,discrete:time_of_day:between9and15,discrete:time_of_day:between12and18,discrete:time_of_day:between15and21,discrete:time_of_day:between18and24,discrete:time_of_day:between21and3,label_source
0,0.998871,0.010354,0.014193,0.024942,0.993955,0.997345,1.002769,1.327529,6.684558,5.039121,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
1,0.995983,0.006215,-0.002742,0.010230,0.993087,0.996082,0.998644,2.044174,6.684592,5.042789,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
2,0.995643,0.033983,-0.051365,0.098204,0.992721,0.996792,1.000948,0.674969,6.683996,5.042479,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
3,0.996405,0.029301,-0.052448,0.104535,0.993695,0.995952,0.998662,0.807471,6.684142,5.041117,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,2
4,0.997930,0.011721,0.024061,0.040938,0.994116,0.996647,1.000360,0.698102,6.684545,5.043921,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,6


In [15]:

import glob
bas = []
folderpath = '/home/wander/OtherProjects/har_flower/sample_data'
for path in glob.iglob(f'{folderpath}/**.csv', recursive=True):
    config = {
            'df_path': path,
            #'df_path': './sample_data/exp_/fold_0/raw_40.csv',
            #'neurons_1' : 32, 
            #'neurons_2' : 16,
            #'from_saved': ../full_data/exp_/saved_model_fold_0,
            'labels': labels
    }

    har = utils.HAR(config)
    #test_results, ba = har.run()
    model, best_hps, best_epoch, test_results, ba = har.hypertunning()
    bas.append(ba.numpy())
    

print(bas)
    

[]


In [16]:
har.mlp.model.summary()

Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 60)                13620     
                                                                 
 dense_59 (Dense)            (None, 6)                 366       
                                                                 
Total params: 13,986
Trainable params: 13,986
Non-trainable params: 0
_________________________________________________________________


In [17]:
har.mlp.data.x_train

AttributeError: 'MLPMultilabel' object has no attribute 'data'

In [18]:

import glob
bas = []
path = '/home/nonroot/sample_data/0A986513-7828-4D53-AA1F-E02D6DF9561B.features_labels.csv'
config = {
        'df_path': path,
        #'df_path': './sample_data/exp_/fold_0/raw_40.csv',
        #'neurons_1' : 32, 
        #'neurons_2' : 16,
        'from_saved': '../full_data/exp_/saved_model_fold_0',
        'labels': labels
}

har = utils.HAR(config)
#test_results, ba = har.run()
#model, best_hps, best_epoch, test_results, ba = har.hypertunning()
#bas.append(ba.numpy())


#print(bas)
har.mlp.model.summary()  

X train-test shape: (3168, 226) - X test shape: (792, 226)
y train-test shape: (3168, 6) - y test shape: (792, 6)
Model: "sequential_23"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_58 (Dense)            (None, 60)                13620     
                                                                 
 dense_59 (Dense)            (None, 6)                 366       
                                                                 
Total params: 13,986
Trainable params: 13,986
Non-trainable params: 0
_________________________________________________________________
